# Trabajo Final: Explorador IA: Guía Personalizada de Viajes usando OpenAI GPT-4 y DALL-E

## Resumen

El objetivo de este proyecto es desarrollar un sistema que genere un perfil de viajero basado en sus gustos y preferencias de viaje y dado un detino, periodo de tiempo y presupuesto devuelven itinerarios personalizados por día. Sugerencias gastronómicas, actividades locales únicas, tips de seguridad y transporte y opcionalemente frases útiles en el idioma local.
En esta POC Utilizaremos la API de OpenAI para GPT-4 para generar una recomendacion de itinerario luego la API de DALL-E generar una imagen descriptiva.

## Índice

1. Introducción
2. Objetivos
3. Metodología
4. Herramientas y Tecnologías
5. Implementación
6. Resultados
7. Conclusiones
8. Referencias

### 1. Introducción

Muchas personas que planifican viajes no encuentran recomendaciones adaptadas a sus intereses específicos (aventura, comida, arte, descanso, etc.). Las guías tradicionales son generales y no personalizadas.
Esto puede generar pérdida de tiempo, frustración o experiencias poco satisfactorias.

### 2. Objetivos

- Generar una descripción textual que devuelven itinerarios personalizados por día, sugerencias gastronómicas, actividades locales únicas, tips de seguridad y transporte.
Opcional: frases útiles en el idioma local.

- Convertir la descripción textual en una imagen utilizando DALL-E.
    •	Prompts que ilustran los lugares sugeridos en la guía (playas, calles, comida, actividades).
    •	Imágenes para representar distintos tipos de viajeros (mochilero, familia, pareja, etc.).
  
### 3. Metodología

1. Recopilación de Datos: Obtener información sobre el perfil del viajero (intereses, presupuesto, clima deseado, duración del viaje, si viaja solo o acompañado).
2. Procesamiento de Texto: Utilizar GPT-4 para generar una descripción textual de itinerarios personalizados sugeridos.
3. Generación de Imagen: Utilizar DALL-E para convertir la descripción en una imagen.

### 4. Herramientas y Tecnologías

- Python
- OpenAI GPT-4 API
- OpenAI DALL-E API



### 5. Implementación
    1. Se deben insalar las bibliotecas de Phyton ejecutando desde una celda:
    •	!pip install python-dotenv
    •	!pip install openai requests
    •	openai-0.28.0
    •	openai-1.84.0
    2. Se debe crear un archivo .env en el directorio de trabajo local con el siguiente contenido:
- API_KEY_GEMINI=tu_api_key_aqui  
- OPENAI_API_KEY=tu_api_key_aqui



In [24]:
#!pip install python-dotenv
#!pip install openai requests
#!pip install --upgrade openai

   ---------------------------------------- 0.0/725.5 kB ? eta -:--:--
   --------------------------------------- 725.5/725.5 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [45]:
import os
from dotenv import load_dotenv
import openai
import requests

load_dotenv()  # Carga el archivo .env
OPENAI_API_KEY  = os.getenv("OPENAI_API_KEY ")

print("API Key cargada:", API_KEY[:4] + "..." if API_KEY else "No se pudo cargar.")
openai.api_key = API_KEY
client = openai.OpenAI() # Usa api_key automáticamente desde la variable de entorno OPENAI_API_KEY


API Key cargada: sk-s...


In [61]:
def openai_chat_gpt4(system, prompt):


    response = client.chat.completions.create(
        model="gpt-4.1",
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response.choices[0].message.content

def openai_image_prof(prompt):
    for attempt in range(3):
        try:
            response = client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                size="1024x1024",
                quality="hd",  # Opcional: 'hd' para más detalle
                n=1
            )
            return response.data[0].url
        except RateLimitError:
            print("🚦 Límite de uso alcanzado. Reintentando en 5 segundos...")
            time.sleep(5)
    return "❌ No se pudo generar la imagen tras varios intentos."
    


In [79]:
prompt="""Eres una guia turistico especializado en personalizar viejes para perfiles de viajeros especificos. Deberas proponer una rutina personalizada para un viajero.
La rutina debe especificar que hacer cada dia y debe considerar los tiempos que demanda cada actividad inclusive los tiempos de traslado para que sea realizable en el dia.
Si las actividades incluyen lugares especificos para visitar se debe tener en cuenta sus horarios de apertura y cierre para que la visita sea exitosa.
Si se incluyen traslados se debe indicar que medio de transporte se recomienda para el traslado o si puede hacerse caminando indicando los tiempos aproximados.
Al final de cada dia se debe indicar el costo aproximado en dolares o euros en caso de destinos de europa de las actividades programadas para el dia y los traslados si los hubiere.
Nuestros viajeros deben contar con reserva de alojamiento para aprovechar el dia por tanto antes de iniciar las rutinas diaria debes especificar en que zona seria recomendable que el viajero reserve su alojamiento para aprovechar al maximo los dias considerando un minimo de 3 noches en cada alojamiento.
Evitar repreguntar en la respuesta.
Nuestro viajero tiene las siguientes caracteristicas: """
texto="Sexo: Femenino. Edad: 40. Viaja con: 2 Amigas. Preferencias: Caminar y visitar lugares, mezclarse con la cultura del lugar, gastronomia, lugares tipicos. Cantidad de dias: 5. Destino/Destinos: Austria, Budapest. Presupuesto: 1000 Euros, excluyendo alojamiento, pero incluyendo el costo de traslado. El viajero inicia su paseo al llegar a Viena."
salida= openai_chat_gpt4(prompt,texto)

In [80]:
print(salida)

¡Perfecto! A continuación encontrarás una propuesta de rutina personalizada para ti y tus amigas, considerando tus intereses en caminatas, cultura local y gastronomía, así como una experiencia auténtica en Austria y Budapest. El itinerario equilibra la visita a sitios emblemáticos, tiempo para disfrutar la ciudad a pie y espacios para descubrir la gastronomía y cultura local, optimizando los traslados y el uso del tiempo disponible dentro de tu presupuesto.

---

## **Recomendación de alojamientos**

**Viena:** Reserva tu alojamiento en la zona de Innere Stadt (1er distrito), ideal para moverse caminando a la mayoría de atractivos principales, buena conexión a transporte y ambiente vibrante.
**Budapest:** Zona recomendada: Distrito V (Belváros-Lipótváros), cerca del Danubio, el Parlamento, zonas peatonales y con fácil acceso a los puntos turísticos y gastronómicos.

---

## **Itinerario Día a Día**

### **Día 1: Llegada y primer recorrido por Viena**
- **08:00-12:00:** Llegada a Viena 

In [74]:
imagen= openai_chat_gpt4("Voy a brindarte una lista de actividades y un perfil de viejero devuelveme unicamente el prompt para enviar al modelo dall-e-3 y generar una imagen que represente un momento de ese viaje, especifica en la descripcion que la fisionomia de los viajantes debe ser latina: ",salida+texto)

In [75]:
print(imagen)

Tres mujeres latinas de 48 años de edad, de aspecto alegre y relajado, disfrutan juntas de un desayuno típico vienés en una elegante mesa del histórico Café Central de Viena. El ambiente es cálido y sofisticado, con grandes columnas, techos altos y vitrales, evocando la atmósfera clásica de principios del siglo XX. Afuera, se intuyen calles peatonales llenas de vida europea. Las mujeres visten ropa cómoda y moderna para caminar, se ven entusiasmadas por su día de exploración cultural en la ciudad. Asegúrate de que la fisionomía de las viajeras luzca auténticamente latina.


In [76]:
image_url=openai_image_prof(imagen)
print(image_url)

https://oaidalleapiprodscus.blob.core.windows.net/private/org-E8thXH85ZSj9viQ3YKWloNQw/user-Z7ZhkJcBqnZTXFBboiOVlBBm/img-GwmLpKT0PMUN7Qa9zrIX0w6U.png?st=2025-06-08T13%3A13%3A17Z&se=2025-06-08T15%3A13%3A17Z&sp=r&sv=2024-08-04&sr=b&rscd=inline&rsct=image/png&skoid=475fd488-6c59-44a5-9aa9-31c4db451bea&sktid=a48cca56-e6da-484e-a814-9c849652bcb3&skt=2025-06-08T02%3A17%3A53Z&ske=2025-06-09T02%3A17%3A53Z&sks=b&skv=2024-08-04&sig=aXUA/axbO0JjcfdG2xrIKC2AlFxC44FsCo%2Bw37KPwt0%3D


In [77]:
from PIL import Image
from io import BytesIO
from datetime import datetime

# Crear carpeta "images" si no existe
os.makedirs("images", exist_ok=True)

# Obtener la imagen desde la URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content))

# Mostrar la imagen
image.show()

# Generar nombre con fecha y hora
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
filename = f"images/image_{timestamp}.png"

# Guardar imagen en carpeta "images"
image.save(filename)
print(f"Imagen guardada como {filename}")

Imagen guardada como images/image_20250608_111352.png
